# Comparison using kerchunk / datatree

In [15]:
from datatree import DataTree
from datatree import map_over_subtree
import dask.bag as db
import xarray as xr
import pandas as pd
import dask
import xclim
import thermofeel as tf
import numpy as np
from distributed import Client
from fsspec.implementations.reference import ReferenceFileSystem
import kerchunk
import coiled
from utils import wbgt, load_elev, adjust_pressure

### TO DO: ONLY NEED SSP 245!
# ds = dt['GISS-E2-1-G']['ssp245'].to_dataset()

In [3]:
client = Client(n_workers=8)

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53637,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:53668,Total threads: 1
Dashboard: http://127.0.0.1:53671/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:53640,


In [ ]:
# # coiled.list_instance_types(backend="aws",arch="x86_64", cores=2, memory="2 Gib")
# import coiled

# coiled.create_software_environment(
#     name="kerchunk-nasa-nex",
#     container="quay.io/carbonplan/kerchunk-nasa-nex",
# )
# cluster = coiled.Cluster(n_workers=8, worker_vm_types=['t3.small'])
# client = cluster.get_client()

In [6]:
# storage_options={"remote_protocol":"s3","target_protocol":"s3", "lazy":True, "skip_instance_cache": True} # options passed to fsspec
# open_dataset_options={"chunks": {}} # opens passed to xarray
# ds = xr.open_dataset(catalog['GISS-E2-1-G/ssp245'], engine="kerchunk", storage_options=storage_options, open_dataset_options=open_dataset_options)

In [49]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_nested.csv"
)
# Select only ssp245
ssp245_catalog = cat_df[cat_df["ID"].str.contains("ssp245")]

ssp245_catalog = ssp245_catalog.iloc[0:2]
# Convert the DataFrame into a dictionary
catalog = ssp245_catalog.set_index("ID").T.to_dict("records")[0]

# invalid? float is not iterable - this GCM ref is invalid
# del catalog['GISS-E2-1-G/ssp245']

In [156]:
def load_ref_ds(gcm_scenario: str, url: str):
    storage_options = {
        "remote_protocol": "s3",
        "target_protocol": "s3",
        "lazy": True,
        "skip_instance_cache": True,
    }  # options passed to fsspec
    open_dataset_options = {"chunks": {}}  # opens passed to xarray
    ds = xr.open_dataset(
        url,
        engine="kerchunk",
        storage_options=storage_options,
        open_dataset_options=open_dataset_options,
    )

    if set(
        [
            "hurs",
            "huss",
            "tasmax",
            "tas",
        ]
    ).issubset(set(list(ds))):
        ds = ds[["hurs", "huss", "tasmax", "tas"]]
        # adding the gcm/scenario combo to attrs for later down the pipeline
        ds.attrs["gcm_scenario"] = gcm_scenario
        return {gcm_scenario: ds}


# convert catalog dict to tuples for dask bag starmap
catalog_tuple_list = list(catalog.items())
# Create a bag from tuple catalog
cat_bag = db.from_sequence(catalog_tuple_list)
# apply load_ref_ds to catalog tuple bag
task_bag = cat_bag.starmap(load_ref_ds).compute()
# convert list of dicts to dict
catalog_computed = {k: v for element in task_bag for k, v in element.items()}

In [157]:
dt = DataTree.from_dict(catalog_computed)
elev = load_elev()

In [166]:
def calc_wbgt(ds):
    ds = ds.to_dataset()
    ds = ds.isel(time=slice(0, 1))
    # calculate elevation-adjusted pressure
    ds["ps"] = xr.apply_ufunc(adjust_pressure, ds["tas"], elev, dask="allowed").rename(
        {"elevation": "ps"}
    )["ps"]
    ds["ps"].attrs["units"] = "Pa"
    ds["hurs"] = xclim.indices.relative_humidity(
        tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
    )
    ds["tasmax"].attrs = {}

    # windspeed assumption of 0.5 m/s (approximating shaded/indoor
    # conditions)
    ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
    ds["WBT"] = tf.thermofeel.calculate_wbt(ds["tasmax"] - 273.15, ds["hurs"])

    ds["BGT"] = tf.thermofeel.calculate_bgt(ds["tasmax"], ds["tasmax"], ds["sfcWind"])
    ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
    ds["WBGT"].attrs["units"] = "degC"
    ds = ds[["WBGT"]]
    ds = dask.optimize(ds)[0]
    output = (
        f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
        f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
    )

    return ds.to_zarr(output, consolidated=True, compute=False, mode="w")


ds_list = [ds for ds in dt.leaves if ds.dims]

bag = db.from_sequence(ds_list, npartitions=len(ds_list)).map(calc_wbgt)

In [ ]:
# @map_over_subtree
# def calc_wbgt(ds):
#     ds = ds.isel(time=slice(0,1))
#     # calculate elevation-adjusted pressure
#     ds["ps"] = xr.apply_ufunc(
#         adjust_pressure, ds["tas"], elev, dask="allowed"
#     ).rename({"elevation": "ps"})["ps"]
#     ds["ps"].attrs["units"] = "Pa"
#     ds["hurs"] = xclim.indices.relative_humidity(
#         tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
#     )
#     ds["tasmax"].attrs = {}

#     # windspeed assumption of 0.5 m/s (approximating shaded/indoor
#     # conditions)
#     ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
#     ds["WBT"] = tf.thermofeel.calculate_wbt(
#         ds["tasmax"] - 273.15, ds["hurs"]
#     )

#     ds["BGT"] = tf.thermofeel.calculate_bgt(
#         ds["tasmax"], ds["tasmax"], ds["sfcWind"]
#     )
#     ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
#     ds["WBGT"].attrs["units"] = "degC"
#     ds = ds[["WBGT"]]
#     ds = dask.optimize(ds)[0]
#     output = (
#         f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
#         f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
#     )


#     # ds.to_zarr(output, consolidated=True, mode="w")
#     return ds

# delayed_datatree = calc_wbgt(dt)

In [ ]:
# output = ("s3://carbonplan-scratch/TEMP_NASA_NEX/DATATREE_TEST.zarr")
# delayed_to_zarr = delayed_datatree.to_zarr(output, compute=False)

In [ ]:
# delayed_obs.compute()

In [ ]:
# 38 secs for single time slice and 10 gcms